In [ ]:
import os
os.chdir("../../")


In [ ]:
from src.losses import loss_function,metrics
from src.callbacks import MetricsCallback,CustomSchedule,ReduceLROnThreshold
from src.models import Encoder,Model


In [ ]:
import tensorflow as tf
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import glob
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import pickle as pkl
import numpy as np
import logging
import polars as pl
logger=logging.getLogger()
import sys
from tqdm import tqdm
logger.setLevel(logging.INFO)
from tqdm import tqdm
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
if tf.test.is_gpu_available():
    print("TensorFlow está utilizando una GPU.")
    print("GPU utilizada:", tf.test.gpu_device_name())
else:
    print("TensorFlow no está utilizando una GPU.")
import glob
import matplotlib.pyplot as plt
import warnings
import joblib
import gc
import pickle as pkl
from itertools import groupby

In [ ]:
from src.training import create_datasets,prepare_validation_data,get_inds,read_indices,read_val_indices,reshape_features,create_datasets
from src.utils import truncate_days,sample_normalize,drop_initial_date

In [ ]:
from src.constants import TARGET,TEST_IDS,SIGMA,DATA_PATH
from src.params import (
    LEARNING_RATE,
    STEPS_PER_EPOCH,
    NUM_EPOCHS,
    WARMUP_STEPS,
    GPU_BATCH_SIZE,
    SAMPLE_NORMALIZE,
    DROP_INITIAL_DATE,
    OPTMIZER_BETA1,
    OPTMIZER_BETA2,
    OPTMIZER_EPSILON,
    ONLY_TEST,
    FOLD,
    TRAIN,
    PREDICT,
    CFG,
    DIM,
    TAM_CONSIDER 
)

In [ ]:

features=pkl.load(open(os.path.join(DATA_PATH,"features.pkl"), "rb"))
targets=pkl.load(open(os.path.join(DATA_PATH,"targets.pkl"), "rb"))
steps_=pkl.load(open(os.path.join(DATA_PATH,"steps.pkl"), "rb"))

In [ ]:

events = pd.read_csv(os.path.join(DATA_PATH,f"train_events.csv"))
events_check = pd.read_csv(os. path.join (DATA_PATH,f"train_events.csv")).dropna(
    subset="timestamp"
)
data = pd.read_parquet(
    os.path.join(DATA_PATH,f"data_train.parquet"),
    columns=["series_id", "step", "anglez", "enmo", "minute", "sine", "cosine", "sleep", "timestamp"],
).iloc[:1000]
data["timestamp"] = pd.to_datetime(data["timestamp"])

## train ids are mapped to int values so we need a dict to map back in the final part to comparare with the events dataset
dict_ids = pkl.load(open(f"{DATA_PATH}/dict_ids.pkl", "rb"))

targets, _, ids = joblib.load(f"{DATA_PATH}/train_data.pkl")
targets = {dict_ids[ids[i]]: targets[i] for i in range(len(targets))}

ids_ = np.load(f"{DATA_PATH}/ids.npy", allow_pickle=True).reshape(-1)
fold_ids = pkl.load(open(f"{DATA_PATH}/folds_ids.pkl", "rb"))
ids_=list(dict_ids.values())

In [ ]:
for i in range(len(features)):
  for j in range(len(features[i])):
    features[i][j] = features[i][j].reshape(-1,features[i][j].shape[1]*CFG["patch_size"])
    feats= features[i][j][:,[0,1,4,5,8,9,]]
    sine_= features[i][j][:,[2,6,10,]].mean(axis=1).reshape(-1,1)
    cosine_=features[i][j][:,[3,7,11,]].mean(axis=1).reshape(-1,1)
    feats=np.concatenate([feats,sine_,cosine_],axis=1)
    features[i][j]=feats
    targets[i][j] =  targets[i][j].reshape(-1,CFG["patch_size"],2).mean(axis=1)

In [ ]:

file = f'{DATA_PATH}/Zzzs_train_multi.parquet'
series_id  = pd.read_parquet(file, columns=['series_id'])
series_id = series_id.series_id.unique()
swaped_dict={val:key for key,val in dict_ids.items()}


In [ ]:
id_=0
j=0
df_=data.loc[data.series_id==id_]
df_=truncate_days(df_,id_, events_check, dict_ids)
df_=drop_initial_date( df_)

In [ ]:
#### training
for fold in range(3,4):
  # use just one fold for trainin. only this fold is used to train the final model to submit to competition, so improving here is a must.
  train_ids=[e for e in fold_ids[fold][0] if e not in TEST_IDS]
  val_ids=[e for e in fold_ids[fold][1] if e not in TEST_IDS]+TEST_IDS
  inds,val_inds,arr= get_inds(train_ids,val_ids,features=features)
  len_val= len(val_inds)
  remainder_val=(GPU_BATCH_SIZE - len(val_inds)%GPU_BATCH_SIZE)%GPU_BATCH_SIZE
  print(f"Remainder validation {remainder_val}")
  for i in range(remainder_val):
      val_inds.append(val_inds[-1])

  dataset =tf.data.Dataset.from_tensor_slices(dict(pd.DataFrame(inds)))
  dataset = dataset .map(read_indices).batch( GPU_BATCH_SIZE, drop_remainder=False)


  val_dataset =tf.data.Dataset.from_tensor_slices(dict(pd.DataFrame(val_inds)))

  val_dataset = val_dataset .map(read_val_indices).batch( GPU_BATCH_SIZE, drop_remainder=False)


  ## same as val_dataset but with batch size 1 to prediction
  val_dataset_backup=tf.data.Dataset.from_tensor_slices(dict(pd.DataFrame(val_inds[:len_val])))
  val_dataset__=val_dataset_backup .map(read_val_indices).batch( 1, drop_remainder=False)



  model=Model()
  scheduler=CustomSchedule(LEARNING_RATE, WARMUP_STEPS)
  optimizer=tf.keras.optimizers.Adam(learning_rate=scheduler, beta_1=OPTMIZER_BETA1, beta_2=OPTMIZER_BETA2, epsilon=OPTMIZER_EPSILON)
  model.build(input_shape=(GPU_BATCH_SIZE, CFG['block_size'] // CFG['patch_size'],DIM ))

  model.compile(optimizer=optimizer,loss=loss_function)
  #model.  summary()

  model.fit(dataset,epochs= 10,steps_per_epoch=500,validation_data=val_dataset, callbacks=[MetricsCallback(FOLD=fold),])
  model.save_weights("my_model_weights")
